In [1]:
import scipy.io
import os
import numpy as np
import random
import json
from numpy import *
import torch

In [7]:
# Read again processed files
x_train = np.load("../benchmark/RAW_DATA/VEHICLE/x_train.npy", allow_pickle=True)
y_train = np.load("../benchmark/RAW_DATA/VEHICLE/y_train.npy", allow_pickle=True)
x_test = np.load("../benchmark/RAW_DATA/VEHICLE/x_test.npy", allow_pickle=True)
y_test = np.load("../benchmark/RAW_DATA/VEHICLE/y_test.npy", allow_pickle=True)

x_train.shape, x_test.shape

((32764, 50, 2), (10931, 50, 2))

In [8]:
y_train[:10], y_test[:10]

(array([[0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0]], dtype=object),
 array([[1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1]], dtype=object))

In [ ]:
mat = scipy.io.loadmat('../benchmark/RAW_DATA/VEHICLE/vehicle.mat')

: 

In [ ]:
mat = scipy.io.loadmat('../benchmark/RAW_DATA/VEHICLE/vehicle.mat')
raw_x, raw_y = mat['X'], mat['Y']
raw_x.shape, raw_y.shape, raw_x[0][0].shape, raw_y[0][0].shape

: 

In [ ]:
raw_x[0][0][:15,:].shape

: 

In [ ]:
NUM_USER = 23
def preprocess(x):
    means = np.mean(x, axis=0)
    std = np.std(x, axis=0)
    
    x = (x - means) * 1.0 / std
    where_are_NaNs = isnan(x)
    x[where_are_NaNs] = 0
    return x

def generate_data():
    # X = np.zeros((NUM_USER,100))
    # y = np.zeros((NUM_USER,100))
    # mat = scipy.io.loadmat('../benchmark/RAW_DATA/VEHICLE/data.mat')
    # raw_x, raw_y = mat['X'], mat['Y']
    # print("number of users:", len(raw_x), len(raw_y))
    # print("number of features:", len(raw_x[0][0][0]))

    X_train = []
    Y_train = []
    X_test = []
    Y_test = []
    for i in range(NUM_USER):
        print("{}-th user has {} samples".format(i, len(raw_x[i][0])))
        
        #print(len(raw_x[i][0]) * 0.75)
        x_processed = preprocess(raw_x[i][0])
        train_samples = int(x_processed.shape[0]*0.75)
        x_train = x_processed[:train_samples,:]
        x_test = x_processed[train_samples,:]
        X_train.append(x_train)
        X_test.append(x_test)
        y_temp = raw_y[i][0]
        y_train = y_temp[:train_samples,:]
        y_test = y_temp[train_samples,:]
        Y_train.append(y_train)
        Y_test.append(y_test)
        
        num = 0
        for j in range(len(raw_y[i][0])):
            if raw_y[i][0][j] == 1:
                num += 1
        print("ratio, ", num * 1.0 / len(raw_y[i][0]))
    # return np.array(X), np.array(y)
    # np.save('../benchmark/RAW_DATA/VEHICLE/x_train.npy', np.array(X_train, dtype=object), allow_pickle=True)
    # np.save('../benchmark/RAW_DATA/VEHICLE/x_test.npy', np.array(X_test, dtype=object), allow_pickle=True)
    # np.save('../benchmark/RAW_DATA/VEHICLE/y_train.npy', np.array(Y_train, dtype=object), allow_pickle=True)
    # np.save('../benchmark/RAW_DATA/VEHICLE/y_test.npy', np.array(Y_test, dtype=object), allow_pickle=True)
    
    return X_train, X_test, Y_train, Y_test

: 

In [ ]:
X1, X2, Y1, Y2 = generate_data()

: 

In [ ]:
X1.shape, len(X2)

: 

In [ ]:
np.array(X1).reshape(len(X1), X1[0].shape[0], X1[0].shape[1])

: 

In [ ]:
X1[1].shape

: 

In [ ]:
# all clients -> 1 file

NUM_USER = 23
def preprocess(x):
    means = np.mean(x, axis=0)
    std = np.std(x, axis=0)
    
    x = (x - means) * 1.0 / std
    where_are_NaNs = isnan(x)
    x[where_are_NaNs] = 0
    return x

def generate_data():
    train_samples_each_client = np.zeros(NUM_USER,dtype=int)
    test_samples = 0
    preprocess_X = preprocess(raw_x[0][0])
    train_samples = int(preprocess_X.shape[0]*0.75)
    train_samples_each_client[0] = train_samples
    test_samples += preprocess_X.shape[0] - train_samples
    
    X_train = preprocess_X[:train_samples,:]
    X_test = preprocess_X[train_samples:, :]
    Y_train = raw_y[0][0][:train_samples, :]
    Y_test = raw_y[0][0][train_samples:, :]
    
    for i in range(1,NUM_USER):
        x_processed = preprocess(raw_x[i][0])
        
        train_samples = int(x_processed.shape[0]*0.75)
        train_samples_each_client[i] = train_samples
        test_samples += x_processed.shape[0] - train_samples
        
        x_train = x_processed[:train_samples,:]
        x_test = x_processed[train_samples:,:]
        X_train = np.concatenate((X_train, x_train), axis=0)
        # print(X_test.shape, x_test.shape)
        X_test = np.concatenate((X_test, x_test), axis=0)
        
        y_temp = raw_y[i][0]
        y_train = y_temp[:train_samples,:]
        y_test = y_temp[train_samples:,:]
        Y_train = np.concatenate((Y_train, y_train), axis=0)
        Y_test = np.concatenate((Y_test, y_test), axis=0)
        
        num = 0
        for j in range(len(raw_y[i][0])):
            if raw_y[i][0][j] == 1:
                num += 1
        print("ratio, ", num * 1.0 / len(raw_y[i][0]))
    # return np.array(X), np.array(y)
    Y_train[Y_train == -1] = 0
    Y_test[Y_test == -1] = 0
    print(train_samples_each_client, test_samples)
    np.save('../benchmark/RAW_DATA/VEHICLE/x_train.npy', np.array(np.transpose(X_train.reshape(-1,2,50),(0,2,1)), dtype=object), allow_pickle=True)
    np.save('../benchmark/RAW_DATA/VEHICLE/x_test.npy', np.array(np.transpose(X_test.reshape(-1,2,50),(0,2,1)), dtype=object), allow_pickle=True)
    np.save('../benchmark/RAW_DATA/VEHICLE/y_train.npy', np.array(Y_train, dtype=object), allow_pickle=True)
    np.save('../benchmark/RAW_DATA/VEHICLE/y_test.npy', np.array(Y_test, dtype=object), allow_pickle=True)
    
    return X_train, X_test, Y_train, Y_test, train_samples_each_client.tolist(), test_samples

: 

In [ ]:
X1, X2, Y1, Y2, ls_train_samples, test_samples = generate_data()
X1.shape, X2.shape, Y1.shape, Y2.shape, ls_train_samples, test_samples

: 

In [ ]:
b = np.array(range(30)).reshape(3,10)
c = b.reshape(3,2,5)
c, c.shape

: 

In [ ]:
a = np.array(range(10)).reshape(2,5)
b = np.transpose(a,(1,0))       #(5,2)
b, b.shape

: 

In [ ]:
b[:,0]

: 

In [ ]:
c = np.transpose(b,(1,0))
c.shape

: 

: 

In [ ]:
c = np.transpose(b,(1,0)).reshape(2,5)
c, c.shape

: 

In [ ]:
c[0,:]

: 

In [ ]:
c, c[:,0], c.shape

: 

: 

In [ ]:
res = np.transpose(c,(0,2,1))
res, res.shape

: 

In [ ]:
np.unique(Y1)

: 

In [ ]:
ls_train_samples

: 

In [ ]:
np.random.randint(12, size=1)

: 

In [2]:
NUM_USER = 23
def gen_list_modalities (missing_rate=0.5, num_modalities=2):
    mat_modals = []
    list_modals_tuples = []
    # num_sample_modals = np.zeros(num_modalities)
    for i in range(NUM_USER):
        unimodal_ind = np.random.randint(num_modalities, size=1)
        # print("Uni: ", np.random.randint(num_modalities, size=1))
        modal_list = np.random.binomial(size=num_modalities, n=1, p=1-missing_rate)
        # modal_indexes = np.where(modal_list==1)[0]
        modal_list[unimodal_ind] = 1
        modal_indexes = np.where(modal_list==1)[0]
        # print(modal_indexes)
        list_modals_tuples.append(tuple(modal_indexes))  
        mat_modals.append(modal_list.tolist())
    mat_modals = np.array(mat_modals)
    num_sample_modals = np.sum(mat_modals, axis=0)
    print("Num_sam:", num_sample_modals)
    return list_modals_tuples      

In [3]:
gen_list_modalities(0.7,2)

Num_sam: [15 15]


[(1,),
 (1,),
 (0,),
 (0, 1),
 (0, 1),
 (0, 1),
 (1,),
 (0,),
 (0,),
 (0,),
 (1,),
 (1,),
 (0,),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (1,),
 (1,),
 (0,),
 (0,),
 (0,),
 (1,)]

In [13]:
gen_list_modalities(0.3,2)

Num_sam: [20 19]


[(0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0,),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0,),
 (0, 1),
 (0, 1),
 (0, 1),
 (1,),
 (0,),
 (0, 1),
 (0, 1),
 (0, 1),
 (1,),
 (0, 1),
 (0,),
 (1,),
 (0, 1)]

In [22]:
gen_list_modalities(1,2)

Num_sam: [14  9]


[(1,),
 (0,),
 (0,),
 (1,),
 (1,),
 (0,),
 (1,),
 (1,),
 (0,),
 (0,),
 (0,),
 (0,),
 (0,),
 (0,),
 (1,),
 (0,),
 (0,),
 (0,),
 (0,),
 (0,),
 (1,),
 (1,),
 (1,)]

In [ ]:
x_ptb = np.load("../benchmark/RAW_DATA/PTBXL_REDUCE/x_train.npy")
test_xptb = np.load("../benchmark/RAW_DATA/PTBXL_REDUCE/x_test.npy")
y_ptb = np.load("../benchmark/RAW_DATA/PTBXL_REDUCE/y_train.npy")
# x_ptb.shape


: 

: 

In [ ]:
np.unique(y_ptb)

: 

In [ ]:
x_ptb.shape, test_xptb.shape, y_ptb.shape # (3samples, feature_dim, #modalities)

: 